In [98]:
import numpy as np

In [99]:
class Node(object):
    """
    Base object for all inputs and outputs.
    """
    def __init__(self, value, grad):
        self.value = value
        self.gradient = grad

In [100]:
class SigmoidNode(object):
    """
    Adds a sigmoid non-linearity to a single input
    """
    def forward(self, x):
        self.x = x
        self.output = Node(1/(1 + np.exp(-1 * self.x.value)), 0.0)
        return self.output
        
    def backward(self):
        s = 1/(1 + np.exp(-1 * self.x.value))
        self.x.gradient = (s * (1 - s)) * self.output.gradient

In [101]:
class MultiplyNode(object):
    """
    Multiplies two inputs
    """
    def forward(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = Node(self.x1.value * self.x2.value, 0)
        return self.output
    
    def backward(self):
        self.x1.gradient = self.x2.value * self.output.gradient
        self.x2.gradient = self.x1.value * self.output.gradient

In [102]:
class AddNode(object):    
    """
    Adds two inputs x1 and x2.
    """
    def forward(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = Node(self.x1.value + self.x2.value, 0)
        return self.output
    
    def backward(self):
        self.x1.gradient = 1 * self.output.gradient
        self.x2.gradient = 1 * self.output.gradient

In [103]:
def forward_neural_Net():
    # w1 * x1
    w1x1 = w1_mul_x1.forward(w1, x1)
    # w2 * x2
    w2x2 = w2_mul_x2.forward(w2, x2)
    # w1*x1 + w2*x2
    w1x1_w2x2 = w1x1_add_w2x2.forward(w1x1, w2x2)
    # w1*x1 + w2*x2 + b
    w1x1_w2x2_b = w1x1w2x2_add_b.forward(w1x1_w2x2, b)
    # sigmoid(w1*x1 + w2*x2 + b)
    output_1 = sigmoid_out.forward(w1x1_w2x2_b)
    
    output1w3 = w3_mul_output1.forward(w3,output_1)
    
    output1w4 = w4_mul_output1.forward(w4,output_1)
    
    #w3*op1 + w4*op1
    #since the output is same the modified weight for the layer becomes
    # (w3+w4) * op1
    w3output1_w4output1 = w3output1_add_w4output1.forward(output1w3,output1w4)
    w3output1_w4output1_b1 = w3output1w4output1_add_b.forward(w3output1_w4output1, b1)
    
    output = sigmoid_out1.forward(w3output1_w4output1_b1)
    
    return output

In [104]:
def backward_neural_net():
    sigmoid_out1.backward()
    w3output1w4output1_add_b.backward()
    w3output1_add_w4output1.backward()
    w4_mul_output1.backward()
    w3_mul_output1.backward()
    
    sigmoid_out.backward()
    w1x1w2x2_add_b.backward()
    w1x1_add_w2x2.backward()
    w2_mul_x2.backward()
    w1_mul_x1.backward()

In [105]:
w1 = Node(0.1, 0.0)
w2 = Node(0.4, 0.0)
w3 = Node(0.2, 0.0)
w4 = Node(0.3, 0.0)
b = Node(-0.02, 0.0)
b1 = Node(-0.01, 0.0)

alpha = 0.001
x1 = Node(0.3, 0.0)
x2 = Node(1.0, 0.0)
y = 0.475

w1_mul_x1 = MultiplyNode()
w2_mul_x2 = MultiplyNode()
w1x1_add_w2x2 = AddNode()
w1x1w2x2_add_b = AddNode()
sigmoid_out = SigmoidNode()

w3_mul_output1 = MultiplyNode()
w4_mul_output1 = MultiplyNode()
w3output1_add_w4output1 = AddNode()
w3output1w4output1_add_b = AddNode()
sigmoid_out1 = SigmoidNode()

In [106]:
forward_output = forward_neural_Net()
print forward_output.value
for i in range(100000):
    forward_output = forward_neural_Net()
    forward_output.gradient = -2 * (y - forward_output.value)
    backward_neural_net()
    w1.value -= alpha * w1.gradient
    w2.value -= alpha * w2.gradient
    w3.value -= alpha * w3.gradient
    w4.value -= alpha * w4.gradient
    b.value -= alpha * b.gradient
    b1.value -= alpha * b1.gradient
    
forward_output.value

0.5721292930904393


0.4750000000518414

# Creating a preceptron class with added method to compute the value of a node at any given instance.


In [107]:
class Perceptron(object):
    def __init__(self, x1, x2, alpha=0.001):
        ### Hyper parameters
        self.alpha = alpha
        ### Initializing weights/bias to a random float between -1 and 1.
        self.w1 = Node(np.random.uniform(-1, 1), 0.0)
        self.w2 = Node(np.random.uniform(-1, 1), 0.0)
        self.b = Node(np.random.uniform(-1, 1), 0.0)
        ### Input and Output variables
        self.x1 = Node(x1, 0.0)
        self.x2 = Node(x2, 0.0)
        ### Initialize operators nodes required 
        ### for processing the inputs within a perceptron
        self.initialize_operators()
    
    def initialize_operators(self):
        self.w1_mul_x1 = MultiplyNode()
        self.w2_mul_x2 = MultiplyNode()
        self.w1x1_add_w2x2 = AddNode()
        self.w1x1_w2x2_add_b = AddNode()
        self.sigmoid = SigmoidNode()
    
    def forward(self):
        w1x1 = self.w1_mul_x1.forward(self.w1, self.x1)
        w2x2 = self.w2_mul_x2.forward(self.w2, self.x2)
        w1x1_w2x2 = self.w1x1_add_w2x2.forward(w1x1, w2x2)
        w1x1_w2x2_b = self.w1x1_w2x2_add_b.forward(w1x1_w2x2, self.b)
        self.sigmoid.forward(w1x1_w2x2_b)
        
    def backward(self):
        self.sigmoid.backward()
        self.w1x1_w2x2_add_b.backward()
        self.w1x1_add_w2x2.backward()
        self.w2_mul_x2.backward()
        self.w1_mul_x1.backward()
    
    def update(self):
        self.w1.value -= self.alpha * self.w1.gradient
        self.w2.value -= self.alpha * self.w2.gradient
        self.b.value -= self.alpha * self.b.gradient
    
    def getValue(self):
        w1x1 = self.w1_mul_x1.forward(self.w1, self.x1)
        w2x2 = self.w2_mul_x2.forward(self.w2, self.x2)
        w1x1_w2x2 = self.w1x1_add_w2x2.forward(w1x1, w2x2)
        w1x1_w2x2_b = self.w1x1_w2x2_add_b.forward(w1x1_w2x2, self.b)
        return self.sigmoid.forward(w1x1_w2x2_b)

The second perceptron is initialized with the outputs of the 1st perceptron and after initialization the whole network is trained to optimize the values of the network.

In [108]:
p = Perceptron(0.11, -1.0, alpha=0.1)

print p.getValue().value
p1 = Perceptron(p.getValue().value,p.getValue().value,alpha=0.01)

# number of iterations
N = 100000
# expected output 
target = 0.3481972639817

for i in range(N):
    # Step 1. Forward Pass
    p.forward()
    p1.forward()
    # Step 2. Calculate Loss. -2 * (y - output) is the gradient of output w.r.t 
    # square loss function.
    p1.sigmoid.output.gradient = -2 * (target - p1.sigmoid.output.value)
    p1.backward()
    p1.update()
    
    p.sigmoid.output.gradient = -2 * (target - p.sigmoid.output.value)
    # Step 3. Backward Pass
    p.backward()
    # Step 4. Update Weights and Bias
    p.update()
    
#     if N%500 == 0:
#         print "p : ",p.getValue().value
#         print "p1 : ", p1.getValue().value

print p1.getValue().value
print p.getValue().value

0.7893539772555763
0.3481972639817004
0.3481972639817004
0.3481972639817001
